In [1]:
import pandas as pd
from census_api import census_df
from scrape_realtor import realtor_df

/Users/miguelrondon/Desktop/CODE/Data Projects/Real-Estate-By-Census/scrape_realtor.py:35: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  realtor_df = pd.read_csv(


In [2]:
state_abrevs=pd.read_json('state_abrev.json', dtype="str")
state_abrevs

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA
5,Colorado,Colo.,CO
6,Connecticut,Conn.,CT
7,Delaware,Del.,DE
8,District of Columbia,D.C.,DC
9,Florida,Fla.,FL


In [3]:
if len(realtor_df["month_date_yyyymm"].unique()) == 1: 
    realtor_df["State"] = realtor_df["State"].str.upper().str.replace(" ", "")
    realtor_df["County"] = realtor_df["County"].str.capitalize()
    realtor_df = pd.merge(realtor_df, state_abrevs, how='left',
                      left_on="State", right_on="Code")
    realtor_df["state"] = realtor_df["State_y"]
    realtor_df = realtor_df.drop(['State_x', 'State_y', 'Abbrev'], axis=1)
    realtor_df.columns = realtor_df.columns.str.lower()
    as_of_month_year = realtor_df["month_date_yyyymm"][0]
    realtor_df
else: 
    raise Exception("the realtor dataframe has more than one date in it, it should only have one date. Please review...") 

In [4]:
import datetime

datetime = datetime.datetime.strptime(
    as_of_month_year, "%Y%m").strftime("%b_%Y")



In [5]:
census_df['County'] = census_df['County'].str.replace("County", "")
census_df.columns = census_df.columns.str.lower().str.replace(" ", "_")

print(census_df.head())


     county     state _total: _total:population_of_one_race:  \
0  Autauga    Alabama   58805                          55648   
1  Baldwin    Alabama  231767                         216743   
2  Barbour    Alabama   25223                          24523   
3     Bibb    Alabama   22293                          21534   
4   Blount    Alabama   59134                          55478   

  _total:population_of_one_race:white_alone  \
0                                     42160   
1                                    189399   
2                                     11317   
3                                     16555   
4                                     50663   

  _total:population_of_one_race:black_or_african_american_alone  \
0                                              11445              
1                                              18217              
2                                              11933              
3                                               4413            

In [6]:
from sqlalchemy import create_engine 
from sqlalchemy_utils import database_exists, create_database
from postgres_credentials import alchemycred


engine= create_engine(alchemycred)
if not database_exists(engine.url):
    create_database(engine.url)


In [7]:
#constantly refreshes data when run 
realtor_df.to_sql("county_prices", engine, if_exists='replace')

#Does not refresh data once created. Data remains the same over time. 
census_df.to_sql("census_demographics", engine, if_exists='fail') 

106